In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-03-25"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
10206,2024-03-25,Croácia Premijer Liga,15:00,Split,GKK Sibenka,1.07,7.22,162.5,1.87,1.79,0.0,0.00,0.00,zsPvS5Vh,0.934579,0.138504,0.534759,0.558659,0.073084,101.504,9.213958,0.090774,2.4,1.341641,0.559017,97.28,1.198,0.175983,0.146897,52.0,182.902,72.153144,0.394491,0.6,1.341641,2.236068,294.59,2.270,0.758848,0.334294,-34.0,76,89,1.28,3.31,121.244,170.390,1.049145,0.030912,NaN,-1.67,-0.334,-0.209581,0.768794,0.8,0.031206,0.40,0.080,77.750000,0.000000,0.0,0.000000
10207,2024-03-25,Dinamarca Liga De Basquete,14:30,Bakken Bears,Svendborg,1.08,7.40,183.5,1.87,1.80,0.0,0.00,0.00,84YL0cBF,0.925926,0.135135,0.534759,0.555556,0.061061,111.148,8.362611,0.075239,1.8,1.643168,0.912871,106.09,1.204,0.133154,0.110593,35.0,168.664,65.327038,0.387321,1.2,1.643168,1.369306,131.14,1.890,0.879375,0.465278,8.0,103,83,1.03,1.58,107.420,204.236,1.053989,0.026974,NaN,0.28,0.056,1.428571,0.000000,0.0,0.000000,-0.14,-0.028,-228.571429,0.000000,0.0,0.000000
10208,2024-03-25,Dinamarca Liga De Basquete,15:15,Randers,Horsens,2.32,1.58,173.5,1.87,1.87,0.0,0.00,0.00,l2njqVur,0.431034,0.632911,0.534759,0.534759,0.063946,219.970,127.696097,0.580516,2.4,1.341641,0.559017,211.12,2.342,1.323261,0.565013,16.0,140.046,71.169916,0.508190,3.0,0.000000,0.000000,127.80,1.612,0.799012,0.495665,63.0,91,90,2.32,1.42,163.342,102.240,0.268338,0.000000,NaN,-2.62,-0.524,-2.519084,0.000000,0.0,0.000000,0.77,0.154,3.766234,0.000000,0.0,0.000000
10209,2024-03-25,Eua Cit,20:00,Tarleton,IPFW,1.43,2.92,151.5,1.80,1.95,0.0,0.00,0.00,IL66rTSD,0.699301,0.342466,0.555556,0.512821,0.041766,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0,77,0.00,2.00,0.000,0.000,0.484409,0.056569,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
10210,2024-03-25,Europa Liga Aba,16:00,Crvena zvezda,Mega Basket,1.03,9.55,166.5,1.85,1.85,0.0,0.00,0.00,Q9Iw9OIJ,0.970874,0.104712,0.540541,0.540541,0.075586,100.180,10.438314,0.104196,2.4,1.341641,0.559017,88.56,1.142,0.176409,0.154473,129.0,140.794,33.503676,0.237962,2.4,1.341641,0.559017,134.64,1.630,0.598790,0.367356,39.0,82,88,1.08,1.53,96.600,243.864,1.138856,0.000000,NaN,0.25,0.050,0.600000,0.000000,0.0,0.000000,-1.58,-0.316,-27.056962,0.000000,0.0,0.000000
10211,2024-03-25,Grécia Liga De Basquete,15:15,Panathinaikos,Promitheas,1.05,9.90,156.5,1.86,1.88,0.0,0.00,0.00,EsNVUAFE,0.952381,0.101010,0.537634,0.531915,0.053391,89.518,9.583414,0.107056,2.4,1.341641,0.559017,95.23,1.060,0.029155,0.027504,88.0,166.380,92.356419,0.555093,1.8,1.643168,0.912871,105.84,2.350,1.823335,0.775887,-7.0,89,84,1.07,1.26,93.902,146.048,1.142995,0.007563,NaN,-0.46,-0.092,-0.543478,0.000000,0.0,0.000000,-1.27,-0.254,-35.039370,0.000000,0.0,0.000000
10212,2024-03-25,Itália Série A2,16:30,Cantu,Verona,1.32,3.20,156.5,1.86,1.84,0.0,0.00,0.00,SS1g42dD,0.757576,0.312500,0.537634,0.543478,0.070076,102.676,15.962747,0.155467,1.2,1.643168,1.369306,85.80,1.254,0.112606,0.089797,-4.0,145.358,83.352822,0.573431,1.2,1.643168,1.369306,126.38,1.836,0.998314,0.543744,-19.0,65,89,1.32,1.42,101.990,185.198,0.588213,0.007644,NaN,-1.60,-0.320,-1.000000,0.837499,0.7,-0.137499,-2.09,-0.418,-5.263158,0.000000,0.0,0.000000
10213,2024-03-25,Itália Série A2

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
10214,13:50,Lituânia Lkl,Juventus,Neptunas,1.56,Back Home
10249,10:00,Bósnia E Herzegovina Prvenstvo Bih Feminina,Kozara F,Zrinjski F,2.53,Back Home
